In [72]:
import paramiko
import ast
import pandas as pd
import logging
import logging.config
from sshtunnel import SSHTunnelForwarder
import pymysql

from sqlalchemy import create_engine

In [52]:
USER='postgres'
PASSWORD='77663540'
HOST='localhost'
PORT='5433'
DATABASE='utea'

In [53]:
# ruta de carpera donde estan las llaves
ruta = open('../unidad_local.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')

# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']

# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [54]:
# funcion para ejecutar una script SQL
# estado = 0 para select
# estado = 1 para update
def dbQuery(q, estado = 0):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            if estado==1:
                conn.commit()  # Commit the transaction
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [55]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
    )

In [57]:
df = pd.read_sql(
    "SELECT * FROM drones_control_bio.resumen_control_bio WHERE agrocittca=false", 
    obtener_engine()
)

In [75]:
query_max = 'SELECT MAX(id) maxid FROM informe_caniero'
max_id = dbQuery(query_max)

In [79]:
max_id = int(max_id[0]['maxid'])
max_id

2056

In [58]:
df['nombre'] = df.apply(lambda x: f"{x['cod_canero']} - {x['nombre_canero']}", axis=1)

In [59]:
df = df[['cod_canero', 'nombre', 'fecha', 'link']]

In [60]:
df['estado'] = 'activo'
df['informe_enviado'] = 'si'
df['slug'] = ''
df['tipo_informe_caniero_id'] = 5

In [62]:
print(len(df))
df.head(3)

83


,cod_canero,nombre,fecha,link,estado,informe_enviado,slug,tipo_informe_caniero_id
0,1580,1580 - CALLE DELGADO IVER,2025-09-04,https://drive.google.com/file/d/1Ssc55SZ0ChICL...,activo,si,,5
1,15085,15085 - CRUZ RUTH JAQUELIN B. DE,2025-09-01,https://drive.google.com/file/d/1glvngwFXDy1HA...,activo,si,,5
2,2505,2505 - CRUZ BARRIENTOS CRISTIAN F.,2025-09-01,https://drive.google.com/file/d/1jhNK-zGjsmztv...,activo,si,,5


In [63]:
for index, row in df.iterrows():
    sql_insert = f'''INSERT INTO informe_caniero (
                        codigo_caniero, 
                        nombre_caniero,
                        fecha,
                        enlace,
                        estado,
                        informe_enviado,
                        slug, 
                        tipo_informe_caniero_id,
                        created_at, 
                        updated_at) 
                VALUES (
                        '{row['cod_canero']}', 
                        '{row['nombre']}', 
                        '{row['fecha']}', 
                        '{row['link']}', 
                        '{row['estado']}', 
                        '{row['informe_enviado']}', 
                        '{row['slug']}',
                        {row['tipo_informe_caniero_id']},
                        NOW(),
                        NOW())'''
    print(sql_insert)

INSERT INTO informe_caniero (
                        codigo_caniero, 
                        nombre_caniero,
                        fecha,
                        enlace,
                        estado,
                        informe_enviado,
                        slug, 
                        tipo_informe_caniero_id,
                        created_at, 
                        updated_at) 
                VALUES (
                        '1580', 
                        '1580 - CALLE DELGADO IVER', 
                        '2025-09-04', 
                        'https://drive.google.com/file/d/1Ssc55SZ0ChICLSH1JxtGiUl9TUBg8a47/view?usp=drivesdk', 
                        'activo', 
                        'si', 
                        '',
                        5,
                        NOW(),
                        NOW())
INSERT INTO informe_caniero (
                        codigo_caniero, 
                        nombre_caniero,
                        fecha,
           

NameError: name 'codigo' is not defined